# Agent Executor From Scratch

In this notebook we will create an agent with a search tool. However, at the start we will force the agent to call the search tool (and then let it do whatever it wants after). This is useful when you want to force agents to call particular tools, but still want flexibility of what happens after that.

This examples builds off the base agent executor. It is highly recommended you learn about that executor before going through this notebook. You can find documentation for that example [here](./base.ipynb).

Any modifications of that example are called below with **MODIFICATION**, so if you are looking for the differences you can just search for that.

## Setup

First we need to install the packages required

In [47]:
!pwd
!pip install --quiet -r requirements.txt

/Users/mfreeman/src/threadr/python/lab
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
snowflake-connector-python 3.2.1 requires platformdirs<4.0.0,>=2.6.0, but you have platformdirs 4.2.0 which is incompatible.
snowflake-connector-python 3.2.1 requires urllib3<1.27,>=1.21.1, but you have urllib3 2.0.4 which is incompatible.
datasets 2.17.1 requires fsspec[http]<=2023.10.0,>=2023.1.0, but you have fsspec 2024.2.0 which is incompatible.
predibase 2024.3.6 requires dataclasses-json==0.5.7, but you have dataclasses-json 0.5.14 which is incompatible.
predibase 2024.3.6 requires pydantic==1.10.13, but you have pydantic 2.3.0 which is incompatible.
questionary 2.0.1 requires prompt_toolkit<=3.0.36,>=2.0, but you have prompt-toolkit 3.0.43 which is incompatible.
great-expectations 0.15.50 requires pydantic<2.0,>=1.10.4, but you have pydantic 2.3.0 which is incompatibl

Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

In [85]:
import os
import warnings
import textwrap

#os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
#os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

warnings.filterwarnings("ignore")


Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [109]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key:")

In [110]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = 'neo4j'
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

VECTOR_INDEX_NAME = 'message-embeddings'
VECTOR_NODE_LABEL = 'Message'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

# Create the graph
graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)


In [111]:
graph.refresh_schema()
print(textwrap.fill(graph.schema, 60))

Node properties are the following: User {name:
STRING},Message {content: STRING, platform: STRING,
timestamp: STRING, embedding: LIST},Channel {name: STRING}
Relationship properties are the following: INTERACTED_WITH
{weight: INTEGER},CONNECTION {type: STRING, weight: INTEGER}
The relationships are the following: (:User)-[:SENT]-
>(:Message),(:User)-[:CONNECTION]->(:User),(:User)-
[:INTERACTED_WITH]->(:User),(:Message)-[:POSTED_IN]-
>(:Channel),(:Message)-[:MENTIONED]->(:User)


# Build our custom retrieval query

In [112]:
retrieval_query2 = """
OPTIONAL MATCH (m:Message)-[:SENT]->(u:User),
               (m)-[:POSTED_IN]->(c:Channel),
               (m)-[:MENTIONED]->(mentioned:User)
WITH m, score, collect(u) AS senders, collect(c) AS channels, collect(mentioned) AS mentionedUsers
RETURN m.content AS text,
       score,
       m {.*, embedding: Null, content: Null, senders: senders, channels: channels, mentionedUsers: mentionedUsers} AS metadata
"""

retrieval_query = """
OPTIONAL MATCH (node)<-[:SENT]-(u:User)
WITH node, score, collect(u) AS users
RETURN node.content AS text,
       score, 
       node {.*, embedding: Null, content: Null, sentBy: users} AS metadata
"""

# Setup the Vector Index 

In [113]:
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(
        model="text-embedding-3-small",
        dimensions=1536,
    ),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="message-embeddings",
    node_label="Message",
    text_node_properties=['content', 'platform', 'timestamp'],
    embedding_node_property='embedding',
    retrieval_query=retrieval_query,
)

In [114]:
import re

def similarity_search(query: str, max_results: int = 1000):
    results = vector_index.search(query, max_results=max_results, search_type="similarity")
    print(f"Found {len(results)} results")
    # Print the raw results for debugging
    #print(results)
    # Clean the page_content from each Document in the results
    content_list = [re.sub(r'^\ncontent: |\x02|\x03\d{2}(,\d{2})?|\u2068|\u2069', '', result.page_content.split('\nplatform:')[0].strip()) for result in results]
    # Print the cleaned content list for verification
    #print(content_list)
    return content_list

    

In [115]:
import re

def similarity_search_v2(v2_query: str, max_results: int = 1000):
    my_retriever = vector_index.as_retriever()
    results = my_retriever.get_relevant_documents(v2_query, max_results=max_results)
    
    formatted_messages = []
    
    for result in results:
        # Access the message content directly via the page_content attribute
        message_content = result.page_content
        
        # Assuming metadata is directly accessible and contains the 'sentBy' information
        # Adjust the access pattern if 'metadata' is structured differently
        username = "Unknown"  # Default username
        if hasattr(result, 'metadata') and 'sentBy' in result.metadata:
            sentBy = result.metadata['sentBy']
            if sentBy and isinstance(sentBy, list) and len(sentBy) > 0:
                username = sentBy[0].get('name', "Unknown")
        
        # Clean the message content
        cleaned_message_content = re.sub(r'\x02|\x03\d{2}(,\d{2})?|\u2068|\u2069', '', message_content)
        
        # Format the message in a chat-like format
        chat_format_message = f"<{username}> {cleaned_message_content}"
        formatted_messages.append(chat_format_message)
    
    return formatted_messages


In [116]:
query = "what is leku's favorite food?"
retriever = vector_index.as_retriever()
retriever.get_relevant_documents(query)

[Document(page_content='ffs', metadata={'timestamp': '2024-04-07T16:47:29.812479+00:00', 'platform': 'IRC', 'sentBy': [{'name': 'nem'}]}),
 Document(page_content='i never really went anywhere with my parents', metadata={'timestamp': '2024-04-06T15:01:20.820982+00:00', 'platform': 'IRC', 'sentBy': [{'name': 'leku'}]}),
 Document(page_content='now add it to my url service!', metadata={'timestamp': '2024-04-04T16:49:59.625174+00:00', 'platform': 'IRC', 'sentBy': [{'name': 'Pilate'}]}),
 Document(page_content="oh, i understand this kind of struggling. i don't even kill spiders", metadata={'timestamp': '2024-04-04T16:43:45.167055+00:00', 'platform': 'IRC', 'sentBy': [{'name': 'oo'}]})]

In [117]:
# query = "nude bots?"
query = "what is leku's favorite food?"
response = similarity_search_v2(query, max_results=10)
print(response)

['<nem> ffs', '<leku> i never really went anywhere with my parents', '<Pilate> now add it to my url service!', "<oo> oh, i understand this kind of struggling. i don't even kill spiders"]


# Define the Cypher Generation Prompt
# Here we define the prompt that will be used to generate Cypher queries
# We use example queries to show how to use the schema to the LLM

In [197]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

### Reading messages in chronological order
```
MATCH (m:Message)-[:POSTED_IN]->(chan:Channel {{name: '#!chases'}})
RETURN m.content AS message, datetime(m.timestamp) AS time
ORDER BY time DESC
```

### Indirect Connection Through Shared Channels
```
MATCH (a:User {{name: 'Alice'}})-[:SENT|POSTED_IN]->(m:Message)-[:POSTED_IN]->(chan:Channel)<-[:POSTED_IN]-(m2:Message)<-[:SENT|POSTED_IN]-(b:User {{name: 'Bob'}})
RETURN DISTINCT chan.name AS SharedChannel
```

### Indirect Connection Through Mutual Connections
```
MATCH (a:User {{name: 'Alice'}})-[:INTERACTED_WITH]->(mutual:User)<-[:INTERACTED_WITH]-(b:User {{name: 'Bob'}})
RETURN DISTINCT mutual.name AS MutualFriend
```

### Is Alice friends with Bob?
```
MATCH (a:User {{name: 'Alice'}})-[:INTERACTED_WITH]-(b:User {{name: 'Bob'}})
RETURN a, b
```

### Showing a complete graph

```
MATCH (chan:Channel)-[:POSTED_IN]-(msg:Message)-[:SENT]-(user:User)
OPTIONAL MATCH (msg)-[:MENTIONED]->(mentioned:User)
RETURN chan, user, msg, mentioned
```

### Pagerduty events
MATCH (m:Message)-[:POSTED_IN]->(c:Channel)
WHERE m.platform = "IRC"
AND m.content CONTAINS "PagerDuty"
AND m.timestamp >= "2024-03-07T00:00:00Z"
AND m.timestamp <= "2024-04-07T23:59:59Z"
RETURN m.content AS EventContent


The question is:
{question}"""

In [198]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain import hub

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE,
)

prompt = hub.pull("hwchase17/openai-functions-agent")
#prompt = ChatPromptTemplate.from_template("{chat_history}")

## Upload our prompt to LangSmith
#hub.push("mfreeman451/threadr-cypher-prompt", CHAT_GENERATION_PROMPT)


In [199]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(model="gpt-4-turbo-preview",temperature=0,openai_api_key=OPENAI_API_KEY),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

# Create the Tools

In [200]:
from typing import Dict, Optional, Type, Union
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool

In [201]:
class TemporalSimilaritySearchInput(BaseModel):
    """Input for the Temporal Similarity Search tool."""
    # Define the inputs for your tool. Our query comes from chat users.
    query: str = Field(description="Did Alice mention anything about yesterday's meeting?")

In [202]:
class SimilaritySearchInput(BaseModel):
    """Input for the Similarity Search tool."""
    # Define the inputs for your tool. Our query comes from chat users.
    query: str = Field(description="Did Alice mention anything about the meeting?")
    max_results: int = Field(description="The maximum number of results to return.", default=1000)
    

In [203]:
class SimilaritySearchTool(BaseTool):
    """A tool that serves to retrieve data from a labeled property graph."""
    
    name: str = "similarity_search_tool"
    description: str = "A tool that serves to retrieve data from a labeled property graph."
    max_results: int = 1000
    args_schema: Type[BaseModel] = SimilaritySearchInput
    
    def _run(
        self,
        query: str,
        max_results: int = 1000,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> Union[Dict, str]:
        """Synchronously run the tool."""
        try:
            return similarity_search_v2(query, max_results=max_results)
        except Exception as e:
            return repr(e)

    async def _arun(
        self,
        query: str,
        max_results: int = 1000,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> Union[Dict, str]:
        """Asynchronously run the tool."""
        try:
            return await similarity_search_v2(query, max_results=max_results)
        except Exception as e:
            return repr(e)
    

In [204]:
class CypherQAInput(BaseModel):
    """Input for the CypherQA tool."""
    # Define the inputs for your tool. Our query comes from chat users.
    query: str = Field(description="Who are alice's friends?")
    

class CypherQATool(BaseTool):
    """A Cypher Question-Answering Tool."""

    name: str = "cypherqa_tool"
    description: str = "A tool that serves to retrieve data from a labeled property graph."
    # If your tool requires initialization parameters, define them here.
    # For a placeholder, we might not need any, but you can add as necessary.
    args_schema: Type[BaseModel] = CypherQAInput

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> Union[Dict, str]:
        """Synchronously run the tool."""
        try:
            return cypherChain.run(query)
        except Exception as e:
            # just return an empty dict if there is an error
            #return repr(e)
            return "Not Found"

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> Union[Dict, str]:
        """Asynchronously run the tool."""
        try:
            return await cypherChain.run(query)
        except Exception as e:
            #return repr(e)
            #return {}
            return "Not Found"



## Create the LangChain agent

First, we will create the LangChain agent. For more information on LangChain agents, see [this documentation](https://python.langchain.com/docs/modules/agents/)

In [205]:
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

#tools = [TavilySearchResults(max_results=1)]
# tools = [CypherQATool(), SimilaritySearchTool(), TavilySearchResults(max_results=1)]
tools = [CypherQATool(), SimilaritySearchTool()]

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4-turbo-preview", streaming=True)
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", streaming=False)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

## Define the graph state

We now define the graph state. The state for the traditional LangChain agent has a few attributes:

1. `input`: This is the input string representing the main ask from the user, passed in as input.
2. `chat_history`: This is any previous conversation messages, also passed in as input.
3. `intermediate_steps`: This is list of actions and corresponding observations that the agent takes over time. This is updated each iteration of the agent.
4. `agent_outcome`: This is the response from the agent, either an AgentAction or AgentFinish. The AgentExecutor should finish when this is an AgentFinish, otherwise it should call the requested tools.


In [206]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## Define the nodes

We now need to define a few different nodes in our graph.
In `langgraph`, a node can be either a function or a [runnable](https://python.langchain.com/docs/expression_language/).
There are two main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

We will also need to define some edges.
Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of several paths may be taken.
The path that is taken is not known until that node is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either:
   a. If the agent said to take an action, then the function to invoke tools should be called
   b. If the agent said that it was finished, then it should finish
2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional edge to take.

In [207]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)


# Define the agent
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    print(f"should_continue: {data}")
    
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

**MODIFICATION**

Here we create a node that returns an AgentAction that just calls the Custom Tool  with the input

In [208]:
tools[0].name

'cypherqa_tool'

In [209]:
from langchain_core.agents import AgentActionMessageLog


def first_agent(agent_inputs):
    action = AgentActionMessageLog(
        # We force call this tool
        #tool="tavily_search_results_json",
        tool="cypherqa_tool",
        #tool="pagerduty_tool",
        #tool="similarity_search_tool",
        # We just pass in the `input` key to this tool
        tool_input=agent_inputs["input"],
        #tool_input = {"query": agent_inputs["input"], "name": "SomeName"},
        log="",
        message_log=[],
    )
    return {"agent_outcome": action}

## Define the graph

We can now put it all together and define the graph!

**MODIFICATION**

We now add a new `first_agent` node which we set as the entrypoint.

In [210]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.add_node("first_agent", first_agent)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("first_agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# After the first agent, we want to take an action
workflow.add_edge("first_agent", "action")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [211]:
inputs = {"input": "What does farmr talk about?", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    #print(s)
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='cypherqa_tool', tool_input='What does farmr talk about?', log='', message_log=[])}
----


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (u:User {name: 'farmr'})-[:SENT]->(m:Message)
RETURN m.content AS FarmrMessages

Full Context:
[{'FarmrMessages': 'YEAH!'}, {'FarmrMessages': 'bysin becomes exhibitionist when drinking?'}, {'FarmrMessages': '!'}, {'FarmrMessages': 'sig: how many US presidents have been assassinated?'}, {'FarmrMessages': 'American presidency has caused the deaths of 20% of presidents, both directly and indirectly.'}, {'FarmrMessages': 'is technically the most DANGEROUS too!'}, {'FarmrMessages': 'i fear im too old to "run"'}, {'FarmrMessages': 'NICE!'}, {'FarmrMessages': 'sig: how many "sheets to the wind" do we need bysin to be before we get worried?'}, {'FarmrMessages': 'plz dont drink entire bottle tonight'}]

> Finished chain.
{'intermediate_steps': [(AgentActionMessageLog(tool='cypherqa_tool', tool_

In [150]:
inputs = {"input": "Does farmr know bysin?", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    #print(s)
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='cypherqa_tool', tool_input='Does farmr know bysin?', log='', message_log=[])}
----


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (a:User {name: 'farmr'})-[:INTERACTED_WITH]-(b:User {name: 'bysin'})
RETURN a, b

Full Context:
[{'a': {'name': 'farmr'}, 'b': {'name': 'bysin'}}]

> Finished chain.
{'intermediate_steps': [(AgentActionMessageLog(tool='cypherqa_tool', tool_input='Does farmr know bysin?', log='', message_log=[]), 'Yes, farmr knows bysin.')]}
----
{'agent_outcome': AgentFinish(return_values={'output': 'Yes, the farmer knows Bysin.'}, log='Yes, the farmer knows Bysin.')}
----
should_continue: {'input': 'Does farmr know bysin?', 'chat_history': [], 'agent_outcome': AgentFinish(return_values={'output': 'Yes, the farmer knows Bysin.'}, log='Yes, the farmer knows Bysin.'), 'intermediate_steps': [(AgentActionMessageLog(tool='cypherqa_tool', tool_input='Does farmr know bysin?', log='', message_log=[]), 'Yes, farmr k

In [134]:
inputs = {"input": "How well does farmr know sig? Can you please include a count of your interactions", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    #print(s)
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='cypherqa_tool', tool_input='How well does farmr know sig? Can you please include a count of your interactions', log='', message_log=[])}
----


> Entering new GraphCypherQAChain chain...
{'intermediate_steps': [(AgentActionMessageLog(tool='cypherqa_tool', tool_input='How well does farmr know sig? Can you please include a count of your interactions', log='', message_log=[]), 'Not Found')]}
----
{'agent_outcome': AgentFinish(return_values={'output': 'It appears there was an issue retrieving the specific interactions between "farmr" and "sig" from the available data. Without access to the detailed interaction data between these entities, I cannot provide a precise analysis or count of their interactions. If you have any other questions or need information on a different topic, feel free to ask!'}, log='It appears there was an issue retrieving the specific interactions between "farmr" and "sig" from the available data. Without access to the det

In [ ]:
inputs = {"input": "What do farmr and sig talk about?", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    #print(s)
    print("----")

In [ ]:
inputs = {"input": "What did farmr and sig talk about yesterday? Today's date is Sun Apr 7 2024", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    #print(s)
    print("----")

In [ ]:
inputs = {"input": "What were the top 3 conversations of utmost importance over the last week? Today's date is April 7th 2024", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    #print(s)
    print("----")

In [ ]:
inputs = {"input": "Does bysin from #!chases know farmr?", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    #print(s)
    print("----")

In [213]:
inputs = {"input": "What PagerDuty events have been fired and delivered over the last month and what was the team split? Today's date is Sun apr 7 2024", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    #print(s)
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='cypherqa_tool', tool_input="What PagerDuty events have been fired and delivered over the last month and what was the team split? Today's date is Sun apr 7 2024", log='', message_log=[])}
----


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (m:Message)-[:POSTED_IN]->(c:Channel)
WHERE m.platform = "IRC"
AND m.content CONTAINS "PagerDuty"
AND m.timestamp >= "2024-03-07T00:00:00Z"
AND m.timestamp <= "2024-04-07T23:59:59Z"
RETURN m.content AS EventContent, c.name AS TeamSplit

Full Context:
[{'EventContent': ' [PagerDuty Alert: DevOps Team] ALERT ID: PD-10234 - Severity: Critical - Issue: Database Response Time Spike - Details: The primary customer          database has experienced a 50% increase in response time as of 03:45 AM UTC.  Action Required: Immediate investigation into database performance and  server load.', 'TeamSplit': '#𝓉𝓌𝑒𝓇𝓀𝒾𝓃'}, {'EventContent': '[PagerDuty|DevOps Team] ALERT ID: PD-10234 | Severity: Critica